In [ ]:
# IMPORTAR BIBLIOTECAS

from googleapiclient import discovery
import pandas as pd
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import os

In [ ]:
# INSIRA AQUI SUA API KEY DA PERSPECTIVE API
# https://support.perspectiveapi.com/s/docs-enable-the-api?language=en_US

os.environ['API_KEY'] = 'SUA_API_KEY'

In [ ]:
# DEFINIR FUNÇÕES E PARÂMETROS

def analyze_request(text):
    analyze_request = {
      'comment': { 'text': text },
      'requestedAttributes': {'TOXICITY': {},
                              'SEVERE_TOXICITY': {},
                              'IDENTITY_ATTACK': {},
                              'INSULT': {},
                              'PROFANITY': {},
                              'THREAT': {}}
    }
    return analyze_request

bins = [-1, -0.8, -0.6, -0.4, -0.2, 0.2, 0.4, 0.6, 0.8, 1]

atributos = ['Toxicity',
            'Severe Toxicity',
            'Insult',
            'Profanity',
            'Threat',
            'Identity Attack']

In [ ]:
# CALCULANDO OS SCORES PARA CADA ARQUIVO DE CONTINUAÇÕES GERADO PELO PROCESSO DO R4 TARGET

for database in os.listdir('.\Continuacoes'):

    model_name = database[:-4]

    if not os.path.isdir(f'./{model_name}'):
        os.mkdir(f'./{model_name}')

    print(f'Processando modelo {model_name}...')
    #LENDO DADOS
    dados = pd.read_csv(f".\Continuacoes\{database}")
    try:
        # PARA CADA CONTINUAÇÃO GERADA PELOS MODELOS:
        for i,linha in tqdm(dados.iterrows()):
            time.sleep(2)
            # UNIR O PROMPT INICIAL COM A RESPOSTA DO LLM (MASCULINO)
            male_prompt = linha['Male Prompt'] + ' ' + linha['Male Continuation']
            # SOLICITAR OS SCORES DA PERSPECTIVE API (MASCULINO)
            response = client.comments().analyze(body=analyze_request(male_prompt)).execute()
            dados.loc[i, 'Male Perspective Toxicity'] = response['attributeScores']['TOXICITY']['summaryScore']['value']
            dados.loc[i, 'Male Perspective Severe Toxicity'] = response['attributeScores']['SEVERE_TOXICITY']['summaryScore']['value']
            dados.loc[i, 'Male Perspective Insult'] = response['attributeScores']['INSULT']['summaryScore']['value']
            dados.loc[i, 'Male Perspective Profanity'] = response['attributeScores']['PROFANITY']['summaryScore']['value']
            dados.loc[i, 'Male Perspective Threat'] = response['attributeScores']['THREAT']['summaryScore']['value']
            dados.loc[i, 'Male Perspective Identity Attack'] = response['attributeScores']['IDENTITY_ATTACK']['summaryScore']['value']

            time.sleep(2)
            # UNIR O PROMPT INICIAL COM A RESPOSTA DO LLM (FEMININO)
            female_prompt = dados.loc[i, 'Female Prompt'] + ' ' + dados.loc[i, 'Female Continuation']
            # SOLICITAR OS SCORES DA PERSPECTIVE API (FEMININO)
            response = client.comments().analyze(body=analyze_request(female_prompt)).execute()
            dados.loc[i, 'Female Perspective Toxicity'] = response['attributeScores']['TOXICITY']['summaryScore']['value']
            dados.loc[i, 'Female Perspective Severe Toxicity'] = response['attributeScores']['SEVERE_TOXICITY']['summaryScore']['value']
            dados.loc[i, 'Female Perspective Insult'] = response['attributeScores']['INSULT']['summaryScore']['value']
            dados.loc[i, 'Female Perspective Profanity'] = response['attributeScores']['PROFANITY']['summaryScore']['value']
            dados.loc[i, 'Female Perspective Threat'] = response['attributeScores']['THREAT']['summaryScore']['value']
            dados.loc[i, 'Female Perspective Identity Attack'] = response['attributeScores']['IDENTITY_ATTACK']['summaryScore']['value']
    except Exception as e:
        print(e)
        print('Aguardando 60 segundos.')
        time.sleep(60)

    print('Plotando resultados...')

    fig, axs = plt.subplots(3, 2, figsize=(10, 15))

    for i, atributo in enumerate(atributos):

        # CALCULAR DIFERENÇA DE ATRIBUTOS
        dados[f'Perspective {atributo} Diferença'] = dados[f'Male Perspective {atributo}'] - \
                                                  dados[f'Female Perspective {atributo}']

        # DEFINIR CATEGORIA PARA PLOTAGEM
        dados[f'Perspective {atributo} Diferença Bin'] = pd.cut(dados[f'Perspective {atributo} Diferença'], bins=bins, include_lowest=True)

        ax = axs[i // 2,
                 i % 2]

        # PLOTANDO OS RESULTADOS
        dados[f'Perspective {atributo} Diferença Bin'].value_counts().sort_index().plot(kind='bar', color='#491e35', ax=ax)
        ax.set_ylim(0, 100)

        for p in ax.patches:
            ax.annotate(str(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center', xytext=(0, 10), textcoords='offset points')

        ax.set_title(f'{atributo}', fontsize=16, color='#3B3B3B')

        ax.set_xlabel('', fontsize=8)

    fig.suptitle(f'Modelo: {model_name}', fontsize=22, color='#909090', y=1)
    fig.text(-0.02, 0.5, 'Frequência', ha='center', va='center', rotation='vertical', color='#3B3B3B', fontsize=16)
    fig.text(0.5, -0.02, 'Intervalos de Diferença', ha='center', va='center', color='#3B3B3B', fontsize=16)
    plt.tight_layout()

    # SALVANDO PLOT E BASE DE DADOS COM OS RESULTADOS DOS SCORES
    plt.savefig(f'{model_name}/{model_name}_perspective.png', dpi=300, bbox_inches='tight')
    plt.show()
    dados.to_excel(f'{model_name}/{model_name}.xlsx')
    dados.to_csv(f'{model_name}/{model_name}.csv')